> 推荐阅读：https://www.osgeo.cn/python_gdal_utah_tutorial/

In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

# 基本属性和方法

In [ ]:
# 打开影像
dataset = gdal.Open('data/hangzhouTM_2010_sample.tif',gdal.GA_ReadOnly)

In [ ]:
# 查看影像元数据
dataset.GetMetadata()

In [ ]:
# 查看影像波段数
dataset.RasterCount

In [ ]:
# 查看影像像元分辨率
print("x:",dataset.RasterXSize,"y:",dataset.RasterYSize)

通过以下两个方法，可以确定一副遥感影像在地面上的位置：

In [ ]:
# 投影信息
dataset.GetProjection()

In [ ]:
# 仿射变换系数
# 如果图像不含地理坐标信息，默认返回值是：(0,1,0,0,0,1)
# 左上角点坐标(padfGeoTransform[0],padfGeoTransform[3])；
# padfGeoTransform[1]和[5]是像元宽度和高度(影像在宽度和高度上的分辨率)
# 如果影像是指北的,padfGeoTransform[2]和padfGeoTransform[4]这两个参数的值为0
# 基于仿射变换系数可以计算影像中某一像元的坐标：
# Xp = padfTransform[0] + X*padfTransform[1] + Y*padfTransform[2];
# Yp = padfTransform[3] + X*padfTransform[4] + Y*padfTransform[5];
dataset.GetGeoTransform()

# 波段与波段信息

In [ ]:
# 读入的影像是多波段的，可以从中选择需要的波段（GetRasterBand从1开始计数）
band_1 = dataset.GetRasterBand(1)

In [ ]:
# 波段统计
# 输入approx_ok=1,表示可以使用近似值; force=1,表示强制计算
# 输出一个列表，依次代表最小值, 最大值, 平均值, 标准差
band_1.GetStatistics(1, 1)

In [ ]:
# 也可以使用GetMinimum和GetMaximum来得到最小值和最大值：
print ("MIN =", band_1.GetMinimum())
print ("MAX =", band_1.GetMaximum())

In [ ]:
# GetNoDataValue方法可以用来得到波段设定的nodata值
print ("NO DATA VALUE =", band_1.GetNoDataValue())

### 绘制波段直方图

In [ ]:
%matplotlib widget
def plotHistigram(band):
    dfmin, dfmax, _, _ = band.GetStatistics(1, 1)    # 记录最大最小值
    nBuckets = 125    # 表示直方图统计的份数
    res = band.GetHistogram(dfmin, dfmax, nBuckets)
    plt.plot(np.linspace(dfmin, dfmax, nBuckets),res)  # 通过linspace方法生成X轴

    
plotHistigram(band_1)

# 读写栅格数据

## 读取栅格数据

`ReadAsArray()`函数可以将栅格数据中的像元值以numpy数组的形式读入，该函数还可以通过参数设置读入的规则：

- xoff,yoff ：指定要读取的部分的原点位置（以像元为单位）。
- xsize,ysize ： 指定要读取部分图像的长和宽（以像元为单位）。
- buf_xsize,buf_ysize ：可以在读取出一部分图像后进行缩放。那么就用这两个参数来定义缩放后图像最终的宽和高，gdal将帮你缩放到这个大小。
- buf_type ：可以对读出的数据的类型进行转换（比如原图数据类型是short，你要把它们缩小成byte）

In [ ]:
help(gdal.Band.ReadAsArray)

In [ ]:
# ReadAsArray可以直接读取整个波段，也可以指定某一窗口范围
# 前两个参数 (xoff, yoff, 默认均为0) 决定了窗口左上角的x值和y值，第3第4个参数 (win_xsize, win_ysize) 决定了窗口的大小
# 比如从(100,100)为原点取一个5*5的窗口
data = band_1.ReadAsArray(100,100,5,5)
data

In [ ]:
# 第5第6个参数 (buf_xsize, buf_ysize)可以在取出窗口数据的基础上进行缩放
# 如在以(100,100)为原点取的5*5的窗口的基础上，扩大为10*10
data = band_1.ReadAsArray(100,100,5,5,10,10)
data

In [ ]:
# 如在以(100,100)为原点取的5*5的窗口的基础上，缩小为3*3
data = band_1.ReadAsArray(100,100,5,5,3,3)
data

In [ ]:
# 也可以直接读取整个dataset, 会多嵌套一维numpyArray
data = dataset.ReadAsArray(100,100,5,5)
data

In [ ]:
# 释放内存
data = None

## 创建栅格数据

In [ ]:
# 截取band1中心256*256的数据
sample = band_1.ReadAsArray((dataset.RasterXSize-256)/2,(dataset.RasterXSize-256)/2,256,256)

In [ ]:
driver = gdal.GetDriverByName( 'GTiff' )
dst_filename = 'data/result/clip.tif'    # 需要注意路径是否存在
dst_ds = driver.Create(dst_filename, 256, 256, 1, gdal.GDT_Byte)   # 注意到之前ReadAsArray的结果中，dtype=uint8,对应gdal数据类型为GDT_Byte

In [ ]:
# 复制仿射变换参数和坐标系信息
dst_ds.SetGeoTransform(dataset.GetGeoTransform())
dst_ds.SetProjection(dataset.GetProjection())

In [ ]:
dst_ds.GetProjection()

In [ ]:
dst_ds.GetRasterBand(1).WriteArray(sample)
dst_ds=None

# 栅格处理

## 栅格计算（地图代数）

In [ ]:
b1 = dataset.GetRasterBand(1).ReadAsArray((dataset.RasterXSize-256)/2,(dataset.RasterXSize-256)/2,256,256)
b2 = dataset.GetRasterBand(2).ReadAsArray((dataset.RasterXSize-256)/2,(dataset.RasterXSize-256)/2,256,256)
print(b1.max())
print(b2.max())

In [ ]:
# NumpyArray之间可以直接进行一对一的四则运算，比如我们可以计算两个波段的平均值:

b1 = dataset.GetRasterBand(1).ReadAsArray((dataset.RasterXSize-256)/2,(dataset.RasterXSize-256)/2,256,256)
b2 = dataset.GetRasterBand(2).ReadAsArray((dataset.RasterXSize-256)/2,(dataset.RasterXSize-256)/2,256,256)

bc = (b1+b2)*0.5

# 输出
driver = gdal.GetDriverByName( 'GTiff' )
dst_ds = driver.Create('data/result/bc1.tif', 256, 256, 1, gdal.GDT_Float64)    # 注意如果数据类型不正确会输出全0的影像
dst_ds.SetGeoTransform(dataset.GetGeoTransform())
dst_ds.SetProjection(dataset.GetProjection())
dst_ds.GetRasterBand(1).WriteArray(bc)
dst_ds=None

特别需要注意的是, 在进行地图代数计算时, 务必要注意读入NumpyAaary的数据类型, 避免出现数值溢出而影响计算结果的情况

In [ ]:
# 还是以上文的计算两波段平均值为例, 这次我们给出一个极端情况, 将b1和b2的所有成员分别赋值为255和1
b1[:]=255   
b2[:]=1
print(b1)
print(b2)

In [ ]:
bc = (b1+b2)*0.5
# 理论上bc的所有成员应为(255+1)*0.5=128
# 但实际上b1,b2由像元类型为gdal.GDT_Byte的dataset读取生成, 数据类型为uint8
# 255+1溢出了uint8的取值范围, 被计算为0
print(bc)

In [ ]:
# 要避免这一问题, 建议在进行地图代数前转换参与运算的numpyArray的数据类型
b1 = b1.astype('float32')    # 和C语言中的加法类似,计算中会自动转换到最高级的数据类型
bc = (b1+b2)*0.5
print(bc)

In [ ]:
# 也可以利用比较运算来检测两份栅格数据之间的变化情况（比如土地覆盖类型的变化）

ds1 = gdal.Open('data/2010.tif',gdal.GA_ReadOnly)
b1 = ds1.GetRasterBand(1).ReadAsArray()
ds2 = gdal.Open('data/2020.tif',gdal.GA_ReadOnly)
b2 = ds2.GetRasterBand(1).ReadAsArray()

bc = (b1!=b2)

# 输出0/1的二值栅格，1对应变化的土地
driver = gdal.GetDriverByName( 'GTiff' )
dst_ds = driver.Create('data/result/bc2.tif', 256, 256, 1, gdal.GDT_Byte)
dst_ds.SetGeoTransform(ds1.GetGeoTransform())
dst_ds.SetProjection(ds1.GetProjection())
dst_ds.GetRasterBand(1).WriteArray(bc)
dst_ds=None

## 波段变换

In [ ]:
# gdal_array包相比gdal包更侧重于数据的像元值，而不怎么侧重元数据等信息，因此在做之和像元值有关的操作时使用gdal_array能使结果更加简洁
from osgeo import gdal_array

src_dir = 'data/hangzhouTM_2010_sample.tif'
arr = gdal_array.LoadFile(src_dir)    # 一个函数直接把栅格文件读入到NumpyArray,但是这种方法更耗内存,因为同时读取了不需要的波段
output = gdal_array.SaveArray(arr[[3, 2, 1], :], "data/result/swap2.tif",format="GTiff", prototype=src_dir)    # arr[[3, 2, 1], :]是numpyArray的分片,将2维array的第3,2,1个array提出重新排序，生成新的二维array
                                                                                                               # 该函数使用原栅格的4、3、2波段作为RGB波段，生成假彩色影像，注意numpyArray是从0开始计数的
                                                                                                               # prototype——原型, 直接从原栅格获取分辨率、投影等信息
arr = output = None    # 手动释放内存

## 栅格裁剪

In [ ]:
# 使用geopandas来获取shapfile文件对应的bbox边界
import geopandas as gpd
shp_dir = 'data/xihu.shp'
xihu = gpd.read_file(shp_dir).to_crs('EPSG:32650').unary_union
bbox = xihu.bounds

ds = gdal.Warp('data/result/clip.tif',
               dataset,
               cutlineDSName = shp_dir,     # 裁剪用的shapfile路径
               outputBounds =  bbox,        # 限定栅格裁剪后的范围,否则只会简单地把shapfile外的数据设为nodata,文件尺寸不会减少
               format = 'GTiff',
               dstNodata = -9999)              # select the no data value you like
               
ds=None     #do other stuff with ds object, it is your cropped dataset. in this case we only close the dataset.
 